In [136]:
# !pip install vaderSentiment
# !pip install imblearn
# !pip install xgboost

In [137]:
import numpy as np
import pandas as pd
# from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [9]:
# Health & Personal Care Category
# df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

# Watches Category
df = pd.read_csv('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
df

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960199,US,39508379,REH8UQZAXQS40,B00005OT9I,2153632,Xhilaration&#174; Silver Dial Bangle Watch,Watches,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13
960200,US,44704251,RJ14QPZEOI9P8,B00005QEM6,903687397,Armitron Men's Titanium 3 Eye Sport Analog Chr...,Watches,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08
960201,US,40571775,R2B1G5650WWFCE,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06
960202,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,118389241,Energizer 393 Button Cell Battery,Watches,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960204 entries, 0 to 960203
Data columns (total 15 columns):
marketplace          960204 non-null object
customer_id          960204 non-null int64
review_id            960204 non-null object
product_id           960204 non-null object
product_parent       960204 non-null int64
product_title        960202 non-null object
product_category     960204 non-null object
star_rating          960204 non-null int64
helpful_votes        960204 non-null int64
total_votes          960204 non-null int64
vine                 960204 non-null object
verified_purchase    960204 non-null object
review_headline      960197 non-null object
review_body          960056 non-null object
review_date          960200 non-null object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [13]:
analyzer = SentimentIntensityAnalyzer()

In [14]:
# Return dictionary of VADER sentiment scores for given sentence. Keys: 'neg', 'neu', 'pos', 'compound'
def sentiment_analyzer_scores(sentence, verbose=False):
    score = analyzer.polarity_scores(sentence)
    if verbose: print("{}".format(str(score)))
    
    return score
    
def sentiment_analyzer_scores_neg(sentence):
    return sentiment_analyzer_scores(sentence)['neg']

def sentiment_analyzer_scores_neu(sentence):
    return sentiment_analyzer_scores(sentence)['neu']

def sentiment_analyzer_scores_pos(sentence):
    return sentiment_analyzer_scores(sentence)['pos']

def sentiment_analyzer_scores_compound(sentence):
    return sentiment_analyzer_scores(sentence)['compound']

In [65]:
df_small = df[:20000]

# Deal with missing values for any of the columns we will be using
# df_small['star_rating'] = df_small['star_rating'].astype(int)
df_small['review_headline'] = df_small['review_headline'].fillna('.')
df_small['review_body'] = df_small['review_body'].fillna('.')
df_small['review_fulltext'] = df_small['review_headline'] + '. ' + df_small['review_body']

df_small['neg'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neg)
df_small['neu'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_neu)
df_small['pos'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_pos)
df_small['compound'] = df_small['review_fulltext'].apply(sentiment_analyzer_scores_compound)

df_small.info()

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 20 columns):
marketplace          20000 non-null object
customer_id          20000 non-null int64
review_id            20000 non-null object
product_id           20000 non-null object
product_parent       20000 non-null int64
product_title        20000 non-null object
product_category     20000 non-null object
star_rating          20000 non-null int64
helpful_votes        20000 non-null int64
total_votes          20000 non-null int64
vine                 20000 non-null object
verified_purchase    20000 non-null object
review_headline      20000 non-null object
review_body          20000 non-null object
review_date          20000 non-null object
review_fulltext      20000 non-null object
neg                  20000 non-null float64
neu                  20000 non-null float64
pos                  20000 non-null float64
compound             20000 non-null float64
dtypes: float64(4), int64(5), ob

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [66]:
df_small['star_rating'].value_counts()
# df_small['verified_purchase'].value_counts()

5    12114
4     3316
1     1940
3     1579
2     1051
Name: star_rating, dtype: int64

In [67]:
df_numer = df_small.select_dtypes(include='number')
df_numer.info()
df_numer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 9 columns):
customer_id       20000 non-null int64
product_parent    20000 non-null int64
star_rating       20000 non-null int64
helpful_votes     20000 non-null int64
total_votes       20000 non-null int64
neg               20000 non-null float64
neu               20000 non-null float64
pos               20000 non-null float64
compound          20000 non-null float64
dtypes: float64(4), int64(5)
memory usage: 1.4 MB


,customer_id,product_parent,star_rating,helpful_votes,total_votes,neg,neu,pos,compound
0,3653882,937001370,5,0,0,0.000,0.616,0.384,0.8169
1,14661224,484010722,5,0,0,0.000,0.357,0.643,0.9531
2,27324930,361166390,2,1,1,0.000,1.000,0.000,0.0000
3,7211452,958035625,5,0,0,0.000,0.890,0.110,0.2732
4,12733322,765328221,4,0,0,0.024,0.715,0.261,0.9886
...,...,...,...,...,...,...,...,...,...
19995,32167840,354933056,5,0,0,0.000,0.754,0.246,0.7351
19996,2194392,649128875,4,0,0,0.068,0.777,0.155,0.4215
19997,15315799,811352940,5,0,0,0.000,0.682,0.318,0.4215
19998,40994978,714629498,5,0,0,0.000,0.517,0.483,0.4215


In [78]:
word_tokenizer = RegexpTokenizer(r'\w+')
df_small['review_wordlist'] = df_small['review_fulltext'].map(lambda x: word_tokenizer.tokenize(x.lower()))

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
df_small

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,...,verified_purchase,review_headline,review_body,review_date,review_fulltext,neg,neu,pos,compound,review_wordlist
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,...,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31,Five Stars. Absolutely love this watch! Get co...,0.000,0.616,0.384,0.8169,"[five, stars, absolutely, love, this, watch, g..."
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,...,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31,I love thiswatch it keeps time wonderfully. I ...,0.000,0.357,0.643,0.9531,"[i, love, thiswatch, it, keeps, time, wonderfu..."
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,...,Y,Two Stars,Scratches,2015-08-31,Two Stars. Scratches,0.000,1.000,0.000,0.0000,"[two, stars, scratches]"
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,...,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31,"Five Stars. It works well on me. However, I fo...",0.000,0.890,0.110,0.2732,"[five, stars, it, works, well, on, me, however..."
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,...,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31,"Beautiful face, but cheap sounding links. Beau...",0.024,0.715,0.261,0.9886,"[beautiful, face, but, cheap, sounding, links,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,US,32167840,R25II0FGG4E2V9,B002SSUQFG,354933056,Seiko Men's SNK809 Seiko 5 Automatic Stainless...,Watches,5,0,0,...,Y,Five Stars,This watch has become my every day where it's ...,2015-08-12,Five Stars. This watch has become my every day...,0.000,0.754,0.246,0.7351,"[five, stars, this, watch, has, become, my, ev..."
19996,US,2194392,R2LE55D844PP2M,B004YM2FV2,649128875,Bling Jewelry Plated Classic Round CZ Ladies W...,Watches,4,0,0,...,Y,Four Stars,"pretty watch, cant get links out myself so I w...",2015-08-12,"Four Stars. pretty watch, cant get links out m...",0.068,0.777,0.155,0.4215,"[four, stars, pretty, watch, cant, get, links,..."
19997,US,15315799,R3MB00E4TTUEH2,B011TBJJSM,811352940,Ritche 20mm Waterproof Silicone Rubber Watch S...,Watches,5,0,0,...,Y,Five Stars,Nice band for the money,2015-08-12,Five Stars. Nice band for the money,0.000,0.682,0.318,0.4215,"[five, stars, nice, band, for, the, money]"
19998,US,40994978,R1HZGYP9W05KJQ,B00BXQSHRI,714629498,"Watch Case Press with 12 Nylon Dies, Watch Cas...",Watches,5,0,0,...,Y,Five Stars,Nice one,2015-08-12,Five Stars. Nice one,0.000,0.517,0.483,0.4215,"[five, stars, nice, one]"


In [79]:
df_small['review_wordlist'] = df_small['review_wordlist'].map(lambda x: ' '.join(x))

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
X = df_small.loc[:, ['helpful_votes', 'total_votes', 'neg', 'neu', 'pos', 'compound', 'review_wordlist']]
y = df_small.loc[:, 'star_rating']

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [82]:
# create our training data list - this is a list of strings, with each string being a review

clean_train_data = []

for train_data in X_train['review_wordlist']:
    clean_train_data.append(train_data)

In [83]:
# create test data list

clean_test_data = []

for test_data in X_test['review_wordlist']:
    clean_test_data.append(test_data)

In [84]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our 
# features are single words

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             max_features = 5000,
                             ngram_range = (1, 3))

In [85]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

# convert to array

train_data_features = train_data_features.toarray()

In [86]:
# check shapes

train_data_features.shape, test_data_features.shape

((15000, 5000), (5000, 5000))

In [87]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()
vocab[:200]

['00',
 '00 watch',
 '000',
 '01',
 '10',
 '10 days',
 '10 minutes',
 '10 seconds',
 '10 year',
 '10 year battery',
 '10 year old',
 '10 years',
 '100',
 '1000',
 '100m',
 '11',
 '12',
 '12 00',
 '12 hours',
 '120',
 '13',
 '13 year',
 '13 year old',
 '14',
 '15',
 '15 minutes',
 '15 years',
 '150',
 '16',
 '17',
 '18',
 '19',
 '1st',
 '20',
 '20 minutes',
 '20 years',
 '200',
 '2014',
 '2015',
 '20mm',
 '21',
 '22',
 '22mm',
 '24',
 '24 hour',
 '24 hours',
 '24 hr',
 '25',
 '28',
 '2nd',
 '30',
 '30 day',
 '30 minutes',
 '300',
 '31',
 '34',
 '34 34',
 '34 br',
 '34 button',
 '34 watch',
 '34 watches',
 '34 water',
 '34 water resistant',
 '34 wrist',
 '35',
 '36',
 '360',
 '3rd',
 '40',
 '400',
 '40mm',
 '42',
 '42mm',
 '44mm',
 '45',
 '4th',
 '50',
 '500',
 '50mm',
 '52mm',
 '55',
 '59',
 '60',
 '600',
 '70',
 '75',
 '7750',
 '80',
 '90',
 '99',
 'ability',
 'able',
 'able tell',
 'able use',
 'able wear',
 'absolute',
 'absolutely',
 'absolutely amazing',
 'absolutely beautiful',
 '

In [88]:
y_train.value_counts()

5    9117
4    2521
1    1412
3    1163
2     787
Name: star_rating, dtype: int64

In [93]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train.drop(columns='review_wordlist'), y_train) 
y_train_resampled.value_counts()

5    9117
4    9117
3    9117
2    9117
1    9117
Name: star_rating, dtype: int64

In [94]:
# Test out different classification algorithms
# Logistic Regression

logreg = LogisticRegression(fit_intercept=True, C=1e12, solver='liblinear', )
model_log = logreg.fit(X_train_resampled, y_train_resampled)
model_log

LogisticRegression(C=1000000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
y_hat_test = logreg.predict(X_test.drop(columns='review_wordlist'))
y_hat_train = logreg.predict(X_train_resampled)

In [100]:
# We could subtract the two columns. If values or equal, difference will be zero. Then count number of zeros 
residuals = np.abs(y_train_resampled - y_hat_train)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    18356
1    18101
2     6761
3     2030
4      337
Name: star_rating, dtype: int64
0    0.402676
1    0.397082
2    0.148316
3    0.044532
4    0.007393
Name: star_rating, dtype: float64


In [101]:
residuals = np.abs(y_test - y_hat_test)
print(pd.Series(residuals).value_counts())
print(pd.Series(residuals).value_counts(normalize=True))

0    2632
1    1526
2     459
3     318
4      65
Name: star_rating, dtype: int64
0    0.5264
1    0.3052
2    0.0918
3    0.0636
4    0.0130
Name: star_rating, dtype: float64


In [102]:
# Confusion Matrix for training set - Logistic Regression
cf = confusion_matrix(y_train_resampled, y_hat_train)
cf

array([[5569, 1898,  878,  610,  162],
       [3968, 2136, 1433, 1203,  377],
       [2065, 1670, 1750, 2134, 1498],
       [ 474,  693, 1090, 2751, 4109],
       [ 175,  569,  424, 1799, 6150]])

In [103]:
print(classification_report(y_train_resampled, y_hat_train))

              precision    recall  f1-score   support

           1       0.45      0.61      0.52      9117
           2       0.31      0.23      0.27      9117
           3       0.31      0.19      0.24      9117
           4       0.32      0.30      0.31      9117
           5       0.50      0.67      0.57      9117

    accuracy                           0.40     45585
   macro avg       0.38      0.40      0.38     45585
weighted avg       0.38      0.40      0.38     45585



In [104]:
# ROC Curve

In [129]:
# fit logistic regression model

lr = LogisticRegression(penalty = 'l2')
lr.fit(train_data_features, y_train)

/Users/megarock/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [130]:
lr.score(train_data_features, y_train)

0.8889333333333334

In [105]:
# K Nearest Neighbors

# Instantiate KNeighborsClassifier
clf_knn = KNeighborsClassifier()

# Fit the classifier
clf_knn.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_knn = clf_knn.predict(X_train_resampled)

cm_knn = confusion_matrix(y_train_resampled, y_hat_knn)
cm_knn

array([[8124,  482,  291,  156,   64],
       [1574, 6836,  431,  192,   84],
       [1266,  428, 6674,  363,  386],
       [ 634,  362,  566, 6060, 1495],
       [ 758,  354,  661, 1344, 6000]])

In [106]:
print(classification_report(y_train_resampled, y_hat_knn))

              precision    recall  f1-score   support

           1       0.66      0.89      0.76      9117
           2       0.81      0.75      0.78      9117
           3       0.77      0.73      0.75      9117
           4       0.75      0.66      0.70      9117
           5       0.75      0.66      0.70      9117

    accuracy                           0.74     45585
   macro avg       0.75      0.74      0.74     45585
weighted avg       0.75      0.74      0.74     45585



In [107]:
# Improve model performance by finding the best value for K

In [132]:
clf_knn2 = KNeighborsClassifier()

# Fit the classifier
clf_knn2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_knn2 = clf_knn2.predict(train_data_features)

print(classification_report(y_train, y_hat_knn2))

cm_knn2 = confusion_matrix(y_train, y_hat_knn2)
cm_knn2

              precision    recall  f1-score   support

           1       0.69      0.75      0.72      1412
           2       0.61      0.22      0.32       787
           3       0.63      0.25      0.36      1163
           4       0.52      0.30      0.38      2521
           5       0.76      0.93      0.84      9117

    accuracy                           0.72     15000
   macro avg       0.64      0.49      0.52     15000
weighted avg       0.69      0.72      0.68     15000



array([[1054,   26,   19,   51,  262],
       [ 153,  173,   36,   76,  349],
       [ 127,   33,  292,  159,  552],
       [  91,   26,   68,  758, 1578],
       [ 102,   27,   49,  427, 8512]])

In [108]:
# Decision Trees

clf_dt = DecisionTreeClassifier(criterion='entropy')

clf_dt.fit(X_train_resampled, y_train_resampled)

# Predict on the training set
y_hat_dt = clf_dt.predict(X_train_resampled)

cm_dt = confusion_matrix(y_train_resampled, y_hat_dt)
cm_dt

array([[8136,  970,    4,    3,    4],
       [ 154, 8954,    3,    1,    5],
       [ 122,  891, 7869,  134,  101],
       [  22,  406,   23, 8216,  450],
       [  59,  459,   25,  412, 8162]])

In [109]:
print(classification_report(y_train_resampled, y_hat_dt))

              precision    recall  f1-score   support

           1       0.96      0.89      0.92      9117
           2       0.77      0.98      0.86      9117
           3       0.99      0.86      0.92      9117
           4       0.94      0.90      0.92      9117
           5       0.94      0.90      0.92      9117

    accuracy                           0.91     45585
   macro avg       0.92      0.91      0.91     45585
weighted avg       0.92      0.91      0.91     45585



In [133]:
clf_dt2 = DecisionTreeClassifier(criterion='entropy')

# Fit the classifier
clf_dt2.fit(train_data_features, y_train)

# Predict on the training set
y_hat_dt2 = clf_dt2.predict(train_data_features)

print(classification_report(y_train, y_hat_dt2))

cm_dt2 = confusion_matrix(y_train, y_hat_dt2)
cm_dt2

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1412
           2       0.99      0.98      0.98       787
           3       0.99      0.94      0.97      1163
           4       0.98      0.89      0.93      2521
           5       0.96      1.00      0.98      9117

    accuracy                           0.97     15000
   macro avg       0.98      0.96      0.97     15000
weighted avg       0.97      0.97      0.97     15000



array([[1412,    0,    0,    0,    0],
       [   0,  773,    2,    1,   11],
       [   0,    8, 1095,    5,   55],
       [   0,    3,    3, 2234,  281],
       [   0,    0,    5,   39, 9073]])

In [110]:
# Bagged Trees

# Instantiate a BaggingClassifier
bagged_tree =  BaggingClassifier(DecisionTreeClassifier(criterion='gini', max_leaf_nodes=100), 
                                 n_estimators=20)

# Fit to the training data
bagged_tree.fit(X_train_resampled, y_train_resampled)

bagged_tree.score(X_train_resampled, y_train_resampled)

0.5059778435888999

In [111]:
y_hat_bagged = bagged_tree.predict(X_train_resampled)

cm_bagged = confusion_matrix(y_train_resampled, y_hat_bagged)
cm_bagged

array([[5608, 2090,  907,  416,   96],
       [2304, 4847,  950,  740,  276],
       [1127, 1981, 3460, 1747,  802],
       [ 393,  948, 1214, 3620, 2942],
       [ 196,  656,  595, 2140, 5530]])

In [112]:
print(classification_report(y_train_resampled, y_hat_bagged))

              precision    recall  f1-score   support

           1       0.58      0.62      0.60      9117
           2       0.46      0.53      0.49      9117
           3       0.49      0.38      0.43      9117
           4       0.42      0.40      0.41      9117
           5       0.57      0.61      0.59      9117

    accuracy                           0.51     45585
   macro avg       0.50      0.51      0.50     45585
weighted avg       0.50      0.51      0.50     45585



In [113]:
# Random Forest

In [114]:
# Instantiate and fit a RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, max_depth = 10)
forest.fit(X_train_resampled, y_train_resampled)
forest.score(X_train_resampled, y_train_resampled)

0.6105078424920478

In [115]:
y_hat_forest = forest.predict(X_train_resampled)

cm_forest = confusion_matrix(y_train_resampled, y_hat_forest)
cm_forest

array([[6322, 1717,  662,  326,   90],
       [1399, 6303,  674,  473,  268],
       [ 894, 1655, 4770,  927,  871],
       [ 346,  876,  945, 4316, 2634],
       [ 174,  660,  552, 1612, 6119]])

In [116]:
print(classification_report(y_train_resampled, y_hat_forest))

              precision    recall  f1-score   support

           1       0.69      0.69      0.69      9117
           2       0.56      0.69      0.62      9117
           3       0.63      0.52      0.57      9117
           4       0.56      0.47      0.51      9117
           5       0.61      0.67      0.64      9117

    accuracy                           0.61     45585
   macro avg       0.61      0.61      0.61     45585
weighted avg       0.61      0.61      0.61     45585



In [117]:
# Grid Search / Random Search to find optimal parameters

In [118]:
# SVM / Linear SVC

In [119]:
# Naive Bayes?

In [120]:
# SVC?

In [121]:
# Ensemble Classifiers

In [122]:
# Adaboost
# Instantiate an AdaBoostClassifier
clf_ab = AdaBoostClassifier()

clf_ab.fit(X_train_resampled, y_train_resampled)
clf_ab.score(X_train_resampled, y_train_resampled)

0.4155972359328727

In [123]:
y_hat_ab = clf_ab.predict(X_train_resampled)

cm_ab = confusion_matrix(y_train_resampled, y_hat_ab)
cm_ab

array([[4280, 3412,  961,  345,  119],
       [2533, 4122, 1465,  733,  264],
       [1333, 2956, 2037, 1733, 1058],
       [ 220, 1237, 1449, 2963, 3248],
       [ 156,  687,  641, 2090, 5543]])

In [124]:
print(classification_report(y_train_resampled, y_hat_ab))

              precision    recall  f1-score   support

           1       0.50      0.47      0.49      9117
           2       0.33      0.45      0.38      9117
           3       0.31      0.22      0.26      9117
           4       0.38      0.32      0.35      9117
           5       0.54      0.61      0.57      9117

    accuracy                           0.42     45585
   macro avg       0.41      0.42      0.41     45585
weighted avg       0.41      0.42      0.41     45585



In [125]:
# Instantiate an GradientBoostingClassifier
clf_gb = GradientBoostingClassifier()

clf_gb.fit(X_train_resampled, y_train_resampled)
clf_gb.score(X_train_resampled, y_train_resampled)

0.48616869584293076

In [126]:
y_hat_gb = clf_gb.predict(X_train_resampled)

cm_gb = confusion_matrix(y_train_resampled, y_hat_gb)
cm_gb

array([[4979, 2620, 1013,  405,  100],
       [2030, 4694, 1371,  704,  318],
       [1037, 2536, 2905, 1679,  960],
       [ 258, 1026, 1285, 3693, 2855],
       [ 122,  686,  608, 1810, 5891]])

In [127]:
print(classification_report(y_train_resampled, y_hat_gb))

              precision    recall  f1-score   support

           1       0.59      0.55      0.57      9117
           2       0.41      0.51      0.45      9117
           3       0.40      0.32      0.36      9117
           4       0.45      0.41      0.42      9117
           5       0.58      0.65      0.61      9117

    accuracy                           0.49     45585
   macro avg       0.49      0.49      0.48     45585
weighted avg       0.49      0.49      0.48     45585



In [ ]:
# XGBoost

In [ ]:
# SGD?

In [ ]:
# Kernel Approximation?

In [52]:
# Clustering

from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score

num_clusters = np.arange(2, 20)

# Set number of clusters at initialization time
for n in num_clusters:
    k_means = KMeans(n_clusters=n) 

    # Run the clustering algorithm
    k_means.fit(df_numer) 

    # Generate cluster index values for each row
    cluster_assignments = k_means.predict(df_numer)
    
    print(n, calinski_harabasz_score(df_numer, cluster_assignments))

2 59486.25585177998
3 77674.18325588538
4 96384.21852818847
5 114546.82979016086
6 127600.80444243301
7 141010.59498579273
8 150682.09351094894
9 161561.0248727341
10 166964.61084244298
11 172383.19385670617
12 179948.81991115178
13 182327.44955418463
14 185306.0671946048
15 187196.97618489747
16 187075.00888801456
17 186023.28848008186
18 185515.0192828231
19 184560.6592451715
20 182653.62410723022
21 177354.84392849842
22 175636.95905210843
23 172503.3659090606
24 170951.62031425143
25 169277.9078372501
26 167893.9965466513
27 166426.02562539277
28 163768.2050005787
29 164446.90279474566
30 163345.56115803414
31 163941.37842270927
32 161616.26932644393
33 162014.7044419554
34 162781.62265305087
35 163483.25020053884
36 164121.61442848394
37 165910.86684445763
38 166263.34904917312
39 166967.81410185085
40 169879.35195707544
41 170036.8446218407
42 172317.16871043303
43 173225.85042507053
44 176405.45462941405
45 175951.96694708266
46 179420.62087730106
47 181230.39342520965
48 181994

In [53]:
cluster_assignments

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [54]:
k_means.labels_

array([20, 48, 43, ..., 40, 34, 23], dtype=int32)

In [55]:
# This code builds on the previous example

# Note that we could also pass in k_means.labels_ instead of cluster_assignments
print(calinski_harabasz_score(df_numer, cluster_assignments))

180807.4371004135
